# Как обучить модель c ограничивающими рамками ориентированную на yolov8

---

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/yolov8-keypoint-detection-custom-train)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

Ultralytics YOLOv8 одна из последних  версий модели обнаружения объектов и сегментации изображений YOLO (вы смотрите только один раз), разработанной  Ultralytics.

В этом руководстве будет рассказано, как обучить модель обнаружения с ограничивающими рамками, ориентированную на YOLOv8.

 [полезный материал, сопровождающий данный ноутбук по YOLOv8](https://blog.roboflow.com/train-yolov8-obb-model/).

## Используйте графическое ускорение

Если вы используете этот ноутбук в Google Colab, перейдите в "Редактировать" -> "Настройки ноутбука" -> "Аппаратный ускоритель", установите значение "Графический процессор" и нажмите "Сохранить". Это гарантирует, что в вашем ноутбуке используется графический процессор, что значительно ускорит обучение модели.


## Шаги в этом мануале

В этом руководстве мы рассмотрим:

- Проверку наличия графического процессора в нашей среде Colab
- Установку YOLOv8
- Подготовку набора данных
- Обучаем OBB-модель YOLOv8
- Выполняем логический вывод на основе нашей модели

## Прежде чем вы начнете

Давайте убедимся, что у нас есть доступ к графическому процессору. Для этого мы можем использовать команду "nvidia-smi". В случае возникновения каких-либо проблем перейдите в "Правка" -> "Настройки ноутбука" -> "Аппаратный ускоритель", установите значение "Графический процессор` и нажмите `Сохранить`.

In [ ]:
!nvidia-smi

Tue Oct 29 12:45:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Установка YOLOv8

Для установки YOL0v8 выполняем:

In [ ]:
!pip install ultralytics -q

import ultralytics
ultralytics.checks()

импортируем YOLOv8 в наш Notebook:

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## Roboflow Universe

Ознакомьтесь с Roboflow Universe, хранилищем наборов данных компьютерного зрения с открытым исходным кодом. Вы можете экспортировать любой набор данных, помеченный, например, как сегментация, как набор данных с ориентированными на YOLOv8 ограничивающими рамками для использования при обучении модели с ориентированными на YOLOv8 ограничивающими рамками.


[![Roboflow Universe](https://media.roboflow.com/notebooks/template/uni-banner-frame.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672878480290)](https://universe.roboflow.com/)



## Подготовка пользовательского набора данных

Создание пользовательского набора данных может быть трудоемким процессом. Сбор изображений, их маркировка и экспорт в нужном формате могут занять десятки или даже сотни часов. К счастью, Roboflow делает этот процесс максимально простым и быстрым. Давайте мы покажем вам, как это сделать!

### Шаг 1: Создание проекта

Перед началом работы вам необходимо создать Roboflow [аккаунт](https://app.roboflow.com/login). Как только вы это сделаете, вы сможете создать новый проект в Roboflow [dashboard](https://app.roboflow.com/). Выберете "Object Detection" в качестве типа проекта.

<img src="https://media.roboflow.com/obb-tutorial/create.png" alt="Object detection selected on the Roboflow Create Project pop up" height="300" />

### Шаг 2: Загрузка изображений

Затем добавьте данные в ваш только что созданный проект. Вы можете сделать это через API или через [веб-интерфейс](https://docs.roboflow.com/adding-data/object-detection).

Если вы перетащите каталог с набором данных в поддерживаемом формате, панель управления Roboflow автоматически считает изображения и аннотации вместе.

<img src="https://media.roboflow.com/obb-tutorial/upload.png" alt="Uploading images to Roboflow" height="300" />

### Шаг 3: Разметка

Если у вас есть только изображения, вы можете пометить их ориентированными ограничивающими рамками в [Roboflow Annotate](https://docs.roboflow.com/annotate).

**Чтобы пометить ограничивающую рамку, воспользуйтесь нашим инструментом для создания полигональных аннотаций.**

Вы также можете взять существующий набор данных сегментации из своей рабочей области или Roboflow Universe и экспортировать его в формате OBB.


<img src="https://media.roboflow.com/obb-tutorial/annotate.png" alt="Annotate an image" height="300" />

### шагp 4: Генерирование новой версии датасета

Теперь, когда мы добавили наши изображения и аннотации, мы можем сгенерировать версию набора данных. При создании версии вы можете выбрать предварительную обработку и дополнения. Этот шаг совершенно необязателен, однако он может позволить вам значительно повысить надежность вашей модели.

<img src="https://media.roboflow.com/keypoint/version.png" alt="Generate a dataset version" height="300" />

### Шаг 5: Экспорт набора данных

Как только версия набора данных будет сгенерирована, мы сможем загрузить ее для использования в обучении модели.

![Сгенерировать версию набора данных](https://media.roboflow.com/keypoint/export.png)


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

import roboflow

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("brad-dwyer").project("aerial-solar-panels")
dataset = project.version(6).download("yolov8-obb")

import yaml

with open(f'{dataset.location}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

data['path'] = dataset.location

with open(f'{dataset.location}/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

## Обучите модель обнаружения OBB-объектов YOLOv8

Загрузив наш набор данных, мы теперь можем обучить модель обнаружения OBB-объектов YOLOv8. Запустите приведенный ниже фрагмент кода, чтобы начать обучение вашей модели.:

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-obb.pt')

results = model.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=640)

Ваша модель будет тренироваться в течение 100 эпох (их количество можно изменить, если это необходимо). После тренировки вы можете запустить тестирование вашей модели, используя изображение из вашего тестового набора.

## Протестируйте модель обнаружения объектов OBB

Давайте протестируем нашу модель обнаружения объектов OBB на изображении:

In [ ]:
model = YOLO('runs/obb/train/weights/best.pt')

import os
import random

random_file = random.choice(os.listdir(f"{dataset.location}/test/images"))
file_name = os.path.join(f"{dataset.location}/test/images", random_file)

results = model(file_name)

print(results[0])

Мы можем визуализировать наши предсказания в виде ограничивающего прямоугольника, используя следующий код:

In [ ]:
# !pip install supervision -q

import supervision as sv
import cv2

detections = sv.Detections.from_ultralytics(results[0])

oriented_box_annotator = sv.OrientedBoxAnnotator()
annotated_frame = oriented_box_annotator.annotate(
    scene=cv2.imread(file_name),
    detections=detections
)

sv.plot_image(image=annotated_frame, size=(16, 16))

Наша модель успешно определила расположение солнечных панелей на изображении. Все солнечные панели имеют ориентированную ограничивающую рамку, которая плотно прилегает к панели.

## 🏆 Поздравляем

### Обучающие ресурсы

Компания Roboflow подготовила множество ресурсов, которые могут заинтересовать вас по мере углубления ваших знаний в области компьютерного зрения:

- [Блокноты Roboflow](https://github.com/roboflow/notebooks): Хранилище из более чем 20 блокнотов, в которых рассказывается о том, как обучать пользовательские модели с использованием различных типов моделей, от YOLOv7 до SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Наша библиотека видеороликов с подробным описанием последних достижений в области компьютерного зрения, подробными руководствами, прилагающимися к нашим записным книжкам, и многим другим.
- [Обсуждение Roboflow](https://discuss.roboflow.com/): У вас есть вопрос о том, как что-то сделать в Roboflow? Задайте свой вопрос на нашем дискуссионном форуме.
- [Модели Roboflow](https://roboflow.com): Узнайте о современных моделях и их производительности. Найдите ссылки и учебные пособия, которые помогут вам в изучении.

### Преобразование форматов данных

Roboflow предоставляет бесплатные утилиты для преобразования данных между десятками популярных форматов компьютерного зрения. Ознакомьтесь с [Roboflow Formats](https://roboflow.com/formats), чтобы ознакомиться с руководствами по преобразованию данных из одного формата в другой за несколько кликов.

### Подключите компьютерное зрение к логике вашего проекта

[Шаблоны Roboflow](https://roboflow.com/templates) - это общедоступная галерея фрагментов кода, которые вы можете использовать для подключения компьютерного зрения к логике вашего проекта. Фрагменты кода варьируются от отправки электронных писем после логического вывода до измерения расстояния до объекта между обнаруженными объектами.

## Задание на лабораторную работу
## Часть 1 Детекция объектов на изображениях
- Ознакомиться с возможностью детекции объектов с помощью YOLOv8 (по желанию можно использовать любую другую версию YOLO старше 8й версии)
- На сайте Roboflow подобрать датасет, который подходит для детекции объектов (можно с любого другого ресурса взять аннотированный датасет, либо создать самостоятельно)
- обучите и протестируйте модель на выбранном датасете, визуализируйте выходные данные, подтвердите, что модель работает
- оформите отчет (в отчете необходимо привести краткий обзор версий yolov8 и выше, основные понятия и опрделения, характеристику своего датасета и параметры подключения, результаты работы)
- итоговый ноутбук с своим датасетом и результатами, а также отчет загрузите в openedu

## Часть 2 Детекция объектов на видео
- самостоятельно подготовить аналогичный ноутбук, но для распознавания объектов на видео
- оформить отчет (в обоих случаях отчеты можно подготовить сразу в ноутбуке)
ссылки в помощь:
[первая](https://uproger.com/obuchaem-s-yolov8-na-polzovatelskih-dannyh-yolov8-instrukcziya-po-rabote/)